In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker 
import pycountry_convert as pc
import folium
import branca
from datetime import datetime, timedelta,date
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests

from keras.layers import Input, Dense, Activation, LeakyReLU
from keras import models
from keras.optimizers import RMSprop, Adam

%matplotlib inline

In [158]:
india_data_json = requests.get('https://api.covid19india.org/raw_data.json').json()
#india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()

df_india = pd.io.json.json_normalize(india_data_json['raw_data'])
df_india = df_india.set_index("dateannounced")
india_data_json

{'raw_data': [{'agebracket': '20',
   'backupnotes': 'Student from Wuhan',
   'contractedfromwhichpatientsuspected': '',
   'currentstatus': 'Recovered',
   'dateannounced': '30/01/2020',
   'detectedcity': 'Thrissur',
   'detecteddistrict': 'Thrissur',
   'detectedstate': 'Kerala',
   'estimatedonsetdate': '',
   'gender': 'F',
   'nationality': 'India',
   'notes': 'Travelled from Wuhan',
   'patientnumber': '1',
   'source1': 'https://twitter.com/vijayanpinarayi/status/1222819465143832577',
   'source2': 'https://weather.com/en-IN/india/news/news/2020-02-14-kerala-defeats-coronavirus-indias-three-covid-19-patients-successfully',
   'source3': '',
   'statecode': 'KL',
   'statepatientnumber': 'KL-TS-P1',
   'statuschangedate': '14/02/2020',
   'typeoftransmission': 'Imported'},
  {'agebracket': '',
   'backupnotes': 'Student from Wuhan',
   'contractedfromwhichpatientsuspected': '',
   'currentstatus': 'Recovered',
   'dateannounced': '02/02/2020',
   'detectedcity': 'Alappuzha',
  

In [159]:
df_india
#total.name = "Total"
#pd.DataFrame(total).style.background_gradient(cmap='summer')

,agebracket,backupnotes,contractedfromwhichpatientsuspected,currentstatus,detectedcity,detecteddistrict,detectedstate,estimatedonsetdate,gender,nationality,notes,patientnumber,source1,source2,source3,statecode,statepatientnumber,statuschangedate,typeoftransmission
dateannounced,,,,,,,,,,,,,,,,,,,
30/01/2020,20,Student from Wuhan,,Recovered,Thrissur,Thrissur,Kerala,,F,India,Travelled from Wuhan,1,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,,KL,KL-TS-P1,14/02/2020,Imported
02/02/2020,,Student from Wuhan,,Recovered,Alappuzha,Alappuzha,Kerala,,,India,Travelled from Wuhan,2,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,,KL,KL-AL-P1,14/02/2020,Imported
03/02/2020,,Student from Wuhan,,Recovered,Kasaragod,Kasaragod,Kerala,,,India,Travelled from Wuhan,3,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,KL,KL-KS-P1,14/02/2020,Imported
02/03/2020,45,Travel history to Italy and Austria,,Recovered,East Delhi (Mayur Vihar),East Delhi,Delhi,,M,India,"Travelled from Austria, Italy",4,https://www.indiatoday.in/india/story/not-a-ja...,https://economictimes.indiatimes.com/news/poli...,,DL,DL-P1,15/03/2020,Imported
02/03/2020,24,"Travel history to Dubai, Singapore contact",,Recovered,Hyderabad,Hyderabad,Telangana,,M,India,"Travelled from Dubai to Bangalore on 20th Feb,...",5,https://www.deccanherald.com/national/south/qu...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,TG,TS-P1,02/03/2020,Imported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,,,,,,,,,,,,6003,,,,,,,
,,,,,,,,,,,,6004,,,,,,,
,,,,,,,,,,,,6005,,,,,,,


In [165]:
df_india[df_india['detectedstate']=="Kerala"]

,agebracket,backupnotes,contractedfromwhichpatientsuspected,currentstatus,detectedcity,detecteddistrict,detectedstate,estimatedonsetdate,gender,nationality,notes,patientnumber,source1,source2,source3,statecode,statepatientnumber,statuschangedate,typeoftransmission
dateannounced,,,,,,,,,,,,,,,,,,,
30/01/2020,20,Student from Wuhan,,Recovered,Thrissur,Thrissur,Kerala,,F,India,Travelled from Wuhan,1,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,,KL,KL-TS-P1,14/02/2020,Imported
02/02/2020,,Student from Wuhan,,Recovered,Alappuzha,Alappuzha,Kerala,,,India,Travelled from Wuhan,2,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,,KL,KL-AL-P1,14/02/2020,Imported
03/02/2020,,Student from Wuhan,,Recovered,Kasaragod,Kasaragod,Kerala,,,India,Travelled from Wuhan,3,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,KL,KL-KS-P1,14/02/2020,Imported
08/03/2020,24,Travel from Italy on 29/02/2020 through Doha,,Recovered,Ranni,Pathanamthitta,Kerala,,M,India,Travelled from Italy on 29/02/2020 through Doha,35,https://indianexpress.com/article/india/corona...,https://www.google.com/url?sa=t&rct=j&q=&esrc=...,http://dhs.kerala.gov.in/wp-content/uploads/20...,KL,KL-PT-P1,27/03/2020,Imported
08/03/2020,54,Travel from Italy on 29/02/2020 through Doha,P35,Recovered,Ranni,Pathanamthitta,Kerala,,M,India,Travelled from Italy on 29/02/2020 through Doha,36,https://www.indiatoday.in/india/story/coronavi...,https://www.google.com/url?sa=t&rct=j&q=&esrc=...,http://dhs.kerala.gov.in/wp-content/uploads/20...,KL,KL-PT-P1,29/03/2020,Imported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
08/04/2020,,,,Hospitalized,,Alappuzha,Kerala,,,,,5547,,,,KL,,08/04/2020,
08/04/2020,,,,Hospitalized,,Alappuzha,Kerala,,,,,5548,,,,KL,,08/04/2020,
08/04/2020,,,,Hospitalized,,Pathanamthitta,Kerala,,,,,5549,,,,KL,,08/04/2020,
